In [2]:
import pandas as pd
import numpy as np
import redapy

In [2]:
ruta_l=r"C:\Users\Guillermo\Desktop\Python\05. Lib gestión pública"
## Abrimos el shp de límites departamentales 2017.
# dep17=gpd.read_file(ruta_l+"\\Bases originales\Límites nacionales\\wlimites_depa.shp")
# dep17=dep17[["iddpto","geometry"]].copy()
# dep17=dep17.set_index("iddpto")

# df=pd.read_excel(ruta_l+"\\Bases originales\\identi_INEI.xlsx") ## identificación etnica
# df2=pd.read_excel(ruta_l+"\\Bases originales\\identi_INEI_pro.xlsx",) ## tiene DNI
### df scrapeada por Diego usando el lenguaje Redatam y el portal de redatam
#df3=pd.read_excel(ruta_l+"\\Bases originales\\tablas_redatam.xlsx") ## tiene DNI
### df de ocupaciones económicas
# df4=pd.read_excel(ruta_l+"\\Bases originales\\ocupación_2017.xlsx") ## ocupaciones 2017
### df de edad en años
df5=pd.read_excel(ruta_l+"\\Bases originales\\Edad en años_2017.xlsx") ## edad en años

#### Frecuencias Univariables

In [7]:
redapy.conversion_redatam_matriz?

Signature:
redapy.conversion_redatam_matriz(
    df,
    continuous=False,
    valor_inicio=0,
    intervalo=None,
)
Docstring:
conversion_redatam_matriz(df: 'DataFrame') -> Dataframe

#Docstring:
Permite de organizar los datos descargados de la plataforma de REDATAM - (INEI) en una formato de base datos de dos dimensiones.
La base de datos de ingreso corresponde a un archivo en formato Excel, el cual se descarga luego de la busqueda de variables en REDATAM
La base de datos resultante es un DataFrame con el ubigeo como índice
File:      c:\users\guillermo\desktop\python\05. lib gestión pública\redapy\limpieza_univar.py
Type:      function


In [12]:
%timeit redapy.conversion_redatam_matriz(df5)
%timeit redapy.conversion_redatam_matriz(df5, continuous=True, kind='intervalos', valor_inicio=0, intervalo=18)
%timeit redapy.conversion_redatam_matriz(df5, continuous=True, kind='descriptivos').head()

6.84 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
10.6 s ± 525 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.8 s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
redapy.conversion_redatam_matriz(df5, continuous=True,  kind='intervalos', valor_inicio=0, intervalo=30).head()
redapy.conversion_redatam_matriz(df5, continuous=True, kind='descriptivos').head()

,resp,0-29,30-59,60-89,90-119
ubigeo,ubigeo,,,,
0101,0101,27105,18869,6132,167.0
0102,0102,40394,23499,6403,148.0
0103,0103,12510,9115,2932,92.0
0104,0104,28737,10550,1604,10.0
0105,0105,22843,14782,5900,201.0


### Frecuencias Tablas cruzadas

In [3]:
import os
from redapy.limpieza_univar import list_ubigeo_index
from redapy.limpieza_univar import extrac_freq_ubigeo

import pandas as pd
import numpy as np
import redapy


In [4]:
ruta_l=r"C:\Users\Guillermo\Desktop\Python\05. Lib gestión pública"
dfc1=pd.read_excel(ruta_l+"\\Bases originales\\carac_vivienda_2017.xlsx")

In [5]:
def clean_str(df):
    df=df.copy()
    dic_replace={"Á":"A","É":"E","Í":"I","Ó":"O","Ú":"U","-":"","\/":""}
    df=(
        df.
        assign(resp=lambda df_:df_.resp.
               str.upper().
               str.strip().
               replace(dic_replace, regex=True)
              )
    )
    return df

In [42]:
def clean_columns_answers(df):
    df=df.copy()
    cols=df.iloc[0:1,0:].values.flatten().tolist()
    cols[-1:]=["ubigeo"]
    df.columns=cols
    df=df.iloc[1:].copy()
    return df

def conversion_redatam_matriz(df, continuous=False, kind=None, valor_inicio=0, intervalo=None):
    """
    conversion_redatam_matriz(df: 'DataFrame') -> Dataframe
    
    #Docstring:
    Permite de organizar los datos descargados de la plataforma de REDATAM - (INEI) en una formato de base datos de dos dimensiones.
    La base de datos de ingreso corresponde a un archivo en formato Excel, el cual se descarga luego de la busqueda de variables en REDATAM
    La base de datos resultante es un DataFrame con el ubigeo como índice

    """
    df=dfc1.iloc[:,1:].copy()
    df=(df.
        rename({df.columns[0]:"resp"},axis=1).
        pipe(clean_str)
       )

    ubigeo, list_index_f=list_ubigeo_index(df)
    ubigeos_des=[]
    
    for p in list(range(0,len(list_index_f))): ## loop para extraer y los datos para cada ubigeo
        ubigeos_des.append(df.
                           pipe(extrac_freq_ubigeo,p).
                           pipe(clean_columns_answers)
                          )
    df_f=(pd.concat(ubigeos_des,axis=0))
    
    return df_f

In [53]:
def prove1(df):
    df=df.copy()
    df=pd.DataFrame(df.rename({np.nan:"fila"},axis=1).set_index("fila").stack())
    return df

a=conversion_redatam_matriz(dfc1).groupby("ubigeo").apply(prove1)



In [34]:
p1=pd.DataFrame(df.rename({np.nan:"fila"},axis=1).set_index("fila").stack())
# p1.loc["Red pública de desagüe dentro de la vivienda"]
fil_var=p1.groupby(level=0).size()
col_var=p1.groupby(level=1).size()

In [68]:
a.reset_index()

,ubigeo,fila,level_2,0
0,0101,RED PUBLICA DENTRO DE LA VIVIENDA,Red pública de desagüe dentro de la vivienda,8755
1,0101,RED PUBLICA DENTRO DE LA VIVIENDA,"Red pública de desagüe fuera de la vivienda, p...",354
2,0101,RED PUBLICA DENTRO DE LA VIVIENDA,"Pozo séptico, tanque séptico o biodigestor",210
3,0101,RED PUBLICA DENTRO DE LA VIVIENDA,Letrina (con tratamiento),213
4,0101,RED PUBLICA DENTRO DE LA VIVIENDA,Pozo ciego o negro,910
...,...,...,...,...
17165,2504,VECINO,"Río, acequia, canal o similar",-
17166,2504,VECINO,Campo abierto o al aire libre,-
17167,2504,VECINO,Otro,-
17168,2504,VECINO,Total,1


In [36]:
def pivot_table(df):
    df=df.copy()    
    df=(df.
        rename(index=str.strip).
        reset_index().
        rename({'level_1':'columna', 0:'freq'},axis=1).
        replace("-","0",regex=True).
        assign(freq=lambda df_:df_.freq.astype(int))
       )
    return df

p2=pivot_table(p1).set_index(["fila","columna"]).sort_index()#.plot(kind='bar')

In [37]:
p2

freq
fila                            columna                                                 
CAMION  CISTERNA U OTRO SIMILAR Campo abierto o al aire libre                          2
                                Letrina (con tratamiento)                              2
                                Otro                                                   6
                                Pozo ciego o negro                                    39
                                Pozo séptico, tanque séptico o biodigestor             4
...                                                                                  ...
VECINO                          Red pública de desagüe dentro de la vivienda           2
                                Red pública de desagüe fuera de la vivienda, pe...     0
                                Río, acequia, canal o similar                          0
                                Total                                                 81
                                ubigeo                                               102

[90 rows x 1 columns]

In [625]:
fil_loc=["Río, acequia, lago, laguna","Manantial o puquio","Otro"]
col_loc=["Total","Otro","Río, acequia, canal o similar"]
p2.loc[pd.IndexSlice[fil_loc,col_loc],:]


freq
fila                       columna                            
Río, acequia, lago, laguna Total                           488
                           Otro                              6
                           Río, acequia, canal o similar     9
Manantial o puquio         Total                            92
                           Otro                              1
                           Río, acequia, canal o similar     1
Otro                       Total                            24
                           Otro                              1
                           Río, acequia, canal o similar     0

In [525]:
col_var

Campo abierto o al aire libre                                                 10
Letrina (con tratamiento)                                                     10
Otro                                                                          10
Pozo ciego o negro                                                            10
Pozo séptico, tanque séptico o biodigestor                                    10
Red pública de desagüe dentro de la vivienda                                  10
Red pública de desagüe fuera de la vivienda, pero dentro de la edificación    10
Río, acequia, canal o similar                                                 10
Total                                                                         10
dtype: int64